# 103 - Catchment Metrics

In [14]:
import duckdb
from pathlib import Path
import teehr.queries.duckdb as tqd
import pandas as pd
import geopandas as gpd
import hvplot.pandas

import spatialpandas as spd
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import panel as pn
import datashader as ds
from holoviews.operation.datashader import (
    rasterize,
    shade,
    regrid,
    inspect_points,
    datashade,
    inspect_polygons,
)
import dask.dataframe as dd

In [15]:
CACHE_DIR = Path(Path.home(), "shared", "teehr-workshop")
STUDY_DIR = Path(CACHE_DIR, "post-event-example")
FORCING_ANALYSIS_ASSIM = Path(STUDY_DIR, "timeseries/forcing_analysis_assim/*.parquet") # primary
FORCING_MEDIUM_RANGE = Path(STUDY_DIR, "timeseries/forcing_medium_range/*.parquet") #secondary
CROSSWALK = Path(STUDY_DIR, "geo/huc10_huc10_crosswalk.parquet")
GEOMETRY = Path(STUDY_DIR, "geo/huc10_geometry.parquet")

In [26]:
opts = dict(
    width=1000,
    height=600

)

In [17]:
# ?tqd.get_metrics

In [24]:
%%time
# primary_sum, secondary_sum, bias
measure = "bias"
cat_gdf = tqd.get_metrics(
    primary_filepath=FORCING_ANALYSIS_ASSIM,
    secondary_filepath=FORCING_MEDIUM_RANGE,
    crosswalk_filepath=CROSSWALK,
    group_by=["primary_location_id"],
    order_by=["primary_location_id"],
    include_metrics=[measure],
    filters=[
        {
            "column":  "reference_time",
            "operator": "=",
            "value": "2023-01-02 06:00:00"
        },
        # {
        #     "column":  "primary_location_id",
        #     "operator": "like",
        #     "value": "huc10-18%"
        # },
    ],
    geometry_filepath=GEOMETRY,
    include_geometry=True
)
cat_gdf.head()

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

CPU times: user 12.5 s, sys: 5.41 s, total: 17.9 s
Wall time: 3.56 s


,primary_location_id,bias,geometry
0,huc10-0101000201,-0.040532,"POLYGON ((-69.51128 46.47166, -69.48252 46.458..."
1,huc10-0101000202,-0.045918,"POLYGON ((-69.48833 46.54592, -69.48760 46.537..."
2,huc10-0101000203,-0.040957,"POLYGON ((-69.50506 46.77384, -69.45326 46.754..."
3,huc10-0101000204,-0.049558,"POLYGON ((-69.29883 46.71177, -69.27392 46.703..."
4,huc10-0101000205,-0.050899,"POLYGON ((-69.19892 46.83243, -69.13182 46.785..."


In [27]:
%%time
tiles = gv.tile_sources.OSM
cat_gdf = cat_gdf.to_crs("EPSG:3857")
cat_sdf = spd.GeoDataFrame(cat_gdf)
cat_gv = gv.Polygons(
    cat_sdf,
    crs=ccrs.GOOGLE_MERCATOR,
    vdims=[measure, "primary_location_id"],
)
cat_rast = rasterize(
    cat_gv,
    aggregator=ds.mean(measure),
    precompute=True,
).opts(**opts, colorbar=True, cmap="fire")
cat_rast * tiles

CPU times: user 4.4 s, sys: 37.9 ms, total: 4.44 s
Wall time: 4.44 s


:DynamicMap   []
   :Overlay
      .Image.I :Image   [Longitude,Latitude]   (Longitude_Latitude bias)
      .WMTS.I  :WMTS   [Longitude,Latitude]